In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pickle
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten,Dropout,BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
import cv2 as cv
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import albumentations as A
from torch.utils.data import Dataset, DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
conv_base = ResNet50(
    weights = 'imagenet',
    include_top = False, #Means we are excluding the top FC layers of our model
    input_shape = (224,224,3)
)  
# conv_base.summary()    
conv_base.trainable = False

94765736/94765736 [==============================] - 0s 0us/step


In [5]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Dense(4,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                        

In [8]:
transform = A.Compose([
#     A.Resize(224, 224), 
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
])


labels = ['grassy','rocky','sandy','marshy']
labels_OHE = {'grassy': np.array([1.,0.,0.,0.]),'rocky': np.array([0.,1.,0.,0.]),'sandy': np.array([0.,0.,1.,0.]),'marshy': np.array([0.,0.,0.,1.])}
root = '/kaggle/input/'
class Dataset:
    def __init__(self, path):
        self.path = path
        self.targets = []
        self.images = []
        self.labels = labels
        for label in self.labels:
            goto = path + label + '/' + label
            #if(label == 'grassy'):
                #goto = path + label  + '/' +label[:-1]
            for i in os.listdir(goto):
                self.targets.append(labels_OHE[label])
                img = cv.imread(goto +'/'+i)
                # print(label+'/'+i)
                # img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                # cv2_imshow(img)
                h,w = img.shape[:2]
                H,W = 0,0
                if h<w:
                    H = 224
                    W = int(224*w/h)
                    img = cv.resize(img,[W,H])
                    img = img[:,int(W/2-112):int(W/2+112),:]
                else:
                    W = 224
                    H = int(224*h/w)
                    img = cv.resize(img,[W,H])
                    img = img[H-W:H,:,:]
#                 print(img.shape, h,w)
#                 print(goto +'/'+i)
                
                img = transform(image = img)
#                 print(img)
#                 img = np.resize(img, [224,224,3])

                self.images.append(img['image'])
    def __len__(self):
        return len(self.targets)
    def __getitem__(self,idx):
        return {self.images[idx]:self.targets[idx]}
    def get_XY(self):
        return self.images, self.targets

mySet = Dataset(root)

In [9]:
X , y = mySet.get_XY()
grass = "/content/drive/MyDrive/SIH 2023/grassy"
rock = "/content/drive/MyDrive/SIH 2023/rocky"
sand = "/content/drive/MyDrive/SIH 2023/sandy"
marsh = "/content/drive/MyDrive/SIH 2023/marshy"


datagen = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=True,
    featurewise_std_normalization=True,
    samplewise_std_normalization=True,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=20,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1/255.,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.3,
    interpolation_order=1,
    dtype=None
)
datagen.fit(X)


In [10]:
# model.load_weights()
# train_data = mySet
# train_loader = DataLoader(dataset=train_data,batch_size=4,shuffle=True)
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate = 0.001,weight_decay = 0.05),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)


# model.fit(np.array(X),np.array(y),batch_size = 32,epochs = 1,validation_split = 0.2)
# iterator = datagen.flow(X,y)
# model.fit(datagen.flow(np.array(X), y, batch_size=32, subset = 'training'))



# model.predict(datagen.flow(np.array(X), y,
#          batch_size=1, subset='validation'))

In [11]:
epochs = 1
model.fit(datagen.flow(np.array(X), y, batch_size=32,
         subset='training'),
         validation_data=datagen.flow(np.array(X), y,
         batch_size=8, subset='validation'),
         steps_per_epoch=len(X) / 40, epochs=epochs)

37/37 [==============================] - 24s 465ms/step - loss: 1.6176 - accuracy: 0.5422 - val_loss: 0.6153 - val_accuracy: 0.7982


In [12]:
model.save('model.h5')

In [13]:

model.predict(datagen.flow(np.array(X), y,
         batch_size=1, subset='validation'))*100

446/446 [==============================] - 7s 12ms/step


array([[5.4495613e+01, 6.5138497e+00, 3.8835533e+01, 1.5500027e-01],
       [6.4951233e+01, 3.4695953e+01, 3.7407015e-02, 3.1540868e-01],
       [3.4569595e+01, 6.5364914e+01, 6.5478899e-02, 1.4757590e-05],
       ...,
       [9.7167549e+01, 3.2718748e-01, 2.4242330e+00, 8.1017174e-02],
       [4.1708736e+01, 5.8190865e+01, 8.8871844e-02, 1.1527703e-02],
       [9.9656174e+01, 4.6200817e-03, 5.2288370e-03, 3.3397669e-01]],
      dtype=float32)